# 数据预处理

>为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始， 而不是从那些准备好的张量格式数据开始。 在Python中常用的数据分析工具中，我们通常使用pandas软件包。 像庞大的Python生态系统中的许多其他扩展包一样，pandas可以与张量兼容。 本节我们将简要介绍使用pandas预处理原始数据，并将原始数据转换为张量格式的步骤。

### 1. 读取数据集

举一个例子，我们首先创建一个人工数据集，并存储在CSV（逗号分隔值）文件 ../data/house_tiny.csv中。 以其他格式存储的数据也可以通过类似的方式进行处理。 下面我们将数据集按行写入CSV文件中。

In [193]:
import os

os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要从创建的CSV文件中加载原始数据集，我们导入pandas包并调用read_csv函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。

In [194]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


### 2. 处理缺失值

注意，“NaN”项代表缺失值。 为了处理缺失的数据，典型的方法包括插值法和删除法， 其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。 在这里，我们将考虑插值法。

In [195]:
input, output = data.iloc[:,0:2], data.iloc[:,-1]
input = input.fillna(input.mean())
input

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [196]:
input = pd.get_dummies(data,dummy_na=True)
input

,NumRooms,Price,Alley_Pave,Alley_nan
0,NaN,127500,1,0
1,2.0,106000,0,1
2,4.0,178100,0,1
3,NaN,140000,0,1


### 3. 转换为张量格式

现在inputs和outputs中的所有条目都是数值类型，它们可以转换为张量格式。 当数据采用张量格式后，可以通过在 1.1节中引入的那些张量函数来进一步操作。

In [197]:
import torch
X, y = torch.tensor(input.values), torch.tensor(output.)
X, y

(tensor([[       nan, 1.2750e+05, 1.0000e+00, 0.0000e+00],
         [2.0000e+00, 1.0600e+05, 0.0000e+00, 1.0000e+00],
         [4.0000e+00, 1.7810e+05, 0.0000e+00, 1.0000e+00],
         [       nan, 1.4000e+05, 0.0000e+00, 1.0000e+00]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

### 4. 练习

1.删除缺失值最多的列。
2.将预处理后的数据集转换为张量格式。

In [201]:
def drop_max_nancol(data):
    nan_ser = data.isna().sum(axis=0)
    # return data.drop(columns=nan_ser.index[nan_ser.argmax()])
    return data.drop(columns=nan_ser.idxmax())
    

def drop_max_nanline(data):
    nan_ser = data.isna().sum(axis=1)
    return data.drop(index=nan_ser.idxmax())

def drop_nanline(data,limit):       #删除缺失值大于等于 limit 的行
    nan_ser = data.isna().sum(axis=1)
    delist = []
    for i in range(nan_ser.shape[0]):
        if nan_ser[i] >= limit:
            delist.append(i)
    return data.drop(index=delist)

print(data)
print(drop_max_nancol(data))
print(drop_max_nanline(data))
print(drop_nanline(data,2))

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000
   NumRooms   Price
0       NaN  127500
1       2.0  106000
2       4.0  178100
3       NaN  140000
   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100


In [199]:
new_data = drop_max_nancol(data)
torch.tensor(new_data.)

tensor([[       nan, 1.2750e+05],
        [2.0000e+00, 1.0600e+05],
        [4.0000e+00, 1.7810e+05],
        [       nan, 1.4000e+05]], dtype=torch.float64)